In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df = pd.read_excel("../data/Coffee Shop Sales.xlsx")
df.head()

,transaction_id,transaction_date,transaction_time,transaction_qty,store_id,store_location,product_id,unit_price,product_category,product_type,product_detail
0,1,2023-01-01,07:06:11,2,5,Lower Manhattan,32,3.0,Coffee,Gourmet brewed coffee,Ethiopia Rg
1,2,2023-01-01,07:08:56,2,5,Lower Manhattan,57,3.1,Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg
2,3,2023-01-01,07:14:04,2,5,Lower Manhattan,59,4.5,Drinking Chocolate,Hot chocolate,Dark chocolate Lg
3,4,2023-01-01,07:20:24,1,5,Lower Manhattan,22,2.0,Coffee,Drip coffee,Our Old Time Diner Blend Sm
4,5,2023-01-01,07:22:41,2,5,Lower Manhattan,57,3.1,Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg


In [9]:
df.info()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149116 entries, 0 to 149115
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   transaction_id    149116 non-null  int64         
 1   transaction_date  149116 non-null  datetime64[ns]
 2   transaction_time  149116 non-null  object        
 3   transaction_qty   149116 non-null  int64         
 4   store_id          149116 non-null  int64         
 5   store_location    149116 non-null  object        
 6   product_id        149116 non-null  int64         
 7   unit_price        149116 non-null  float64       
 8   product_category  149116 non-null  object        
 9   product_type      149116 non-null  object        
 10  product_detail    149116 non-null  object        
 11  sales             149116 non-null  float64       
 12  transaction_hour  149116 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(5), object(5)
memor

transaction_id      0
transaction_date    0
transaction_time    0
transaction_qty     0
store_id            0
store_location      0
product_id          0
unit_price          0
product_category    0
product_type        0
product_detail      0
sales               0
transaction_hour    0
dtype: int64

In [ ]:
# Create a new column for total trasaction value (sales)
df["sales"] = df["transaction_qty"] * df["unit_price"]

# I convert the time column to string first to ensure consistency, then calculate hour
df["transaction_hour"] = (
    df["transaction_time"].astype(str).str.split(":").str[0].astype(int)
)

df[
    ["transaction_time", "transaction_hour", "transaction_qty", "unit_price", "sales"]
].head()

,transaction_time,transaction_hour,transaction_qty,unit_price,sales
0,07:06:11,7,2,3.0,6.0
1,07:08:56,7,2,3.1,6.2
2,07:14:04,7,2,4.5,9.0
3,07:20:24,7,1,2.0,2.0
4,07:22:41,7,2,3.1,6.2


In [8]:
# Get summary statistics for numerical columns
# This helps identify outliers (e.g., negative quantity or extreme prices)
df.describe()

,transaction_id,transaction_date,transaction_qty,store_id,product_id,unit_price,sales,transaction_hour
count,149116.000000,149116,149116.000000,149116.000000,149116.000000,149116.000000,149116.000000,149116.000000
mean,74737.371872,2023-04-15 11:50:32.173609984,1.438276,5.342063,47.918607,3.382219,4.686367,11.735790
min,1.000000,2023-01-01 00:00:00,1.000000,3.000000,1.000000,0.800000,0.800000,6.000000
25%,37335.750000,2023-03-06 00:00:00,1.000000,3.000000,33.000000,2.500000,3.000000,9.000000
50%,74727.500000,2023-04-24 00:00:00,1.000000,5.000000,47.000000,3.000000,3.750000,11.000000
75%,112094.250000,2023-05-30 00:00:00,2.000000,8.000000,60.000000,3.750000,6.000000,15.000000
max,149456.000000,2023-06-30 00:00:00,8.000000,8.000000,87.000000,45.000000,360.000000,20.000000
std,43153.600016,NaN,0.542509,2.074241,17.930020,2.658723,4.227099,3.764662


In [ ]:
# how many transactions included multiple items
transaction_counts = df["transaction_id"].value_counts()

# oders with more than 1 item
multi_item_txns = transaction_counts[transaction_counts > 1]

print(f"there are {len(multi_item_txns)} orders that included more than 1 item.")

there are 0 orders that included more than 1 item.
